<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/luiz_gontijo/Aula_8_Exerci%CC%81cio_Template_Luiz_Gontijo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Luiz Fernando da Costa Gontijo'
print(f'Meu nome é {nome}')

Meu nome é Luiz Fernando da Costa Gontijo


# Sobre a realização do trabalho

Foram realizadas as duas implementações sugeridas pelo professor: em loop e em matrizes. Como a implementação em loop demorou muito tempo para ser executada, mesmo com uma pequena amostra da base de dados, optei por não obter seus resultados para a base de dados completa. Dessa forma, os resultados apresentados nesse notebook foram obtidos com a implementação em matrizes. Como forma de estudo, também foram implementados os algoritmos sem a utilização de projeções lineares - tais códigos estão na parte final do notebook.

Foram realizadas etapas de salvamento do modelo obtido a partir da realização de algumas etapas de validação. Esses armazenamentos foram realizados somente se o modelo obtido em alguma etapa obtivesse melhor PPL de validação que a etapa anterior.  

Para continuar o treinamento com maior quantidade de realizações, optei por um treinamento inicial com 400.000.000 exemplos. Esse modelo obtido foi salvo e depois carregado para continuar treinando até o limite disponibilizado pelo Colab. A PPL de teste, assim, foi obtida de acordo com o modelo final encontrado.

A limitação da RAM foi bastante prejudicial para esse trabalho. A alternativa considerada por mim para superar esse obstáculo foi alterar a função de definição do dataset bão armazenando algumas informações em listas. O armazenamento de informação na GPU (com o método .to(device)) foi feito somente para o treinamento com os dados completos. Logo, algumas etapas do notebook original foram alteradas para trabalhar fora da GPU.

Por fim, ao final do notebook também estão alguns rascunhos utilizados ao decorrer da implementação do trabalho.

#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 6.7 MB/s 
     |████████████████████████████████| 596 kB 7.7 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 29.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 17:01:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [4]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokensIds_n = []
        self.y = []
        for text in tqdm_notebook(texts):
            tokens_ids = tokenize(text, tokenizer)
            if len(tokens_ids) < context_size + 1:
                continue
            for i in range(len(tokens_ids)-context_size):
                self.tokensIds_n.append(tokens_ids[i:i+context_size])      
                self.y.append(tokens_ids[i+context_size])
                
    def __len__(self):  
        return len(self.tokensIds_n)

    def __getitem__(self, idx):
        
        return torch.tensor(self.tokensIds_n[idx]).long(), torch.tensor(self.y[idx]).long()

## Testando se a implementação do MyDataset está correta

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [6]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 22:20:43--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.128, 172.217.13.240, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   135MB/s    in 0.9s    

2022-05-25 22:20:44 (135 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [7]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [13]:
# implementação com matrizes

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        self.embedding_layer = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)

        # linear projections
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_0 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        # definir a softmax
        self.softmax1 = nn.Softmax(dim=-1)
        self.softmax2 = nn.Softmax(dim=-1)

        # camada linear - aplicar somente uma camada
        hidden_layer = 64
        self.linear1 = nn.Linear(embedding_dim, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, vocab_size, bias=False)
        self.tanh1 = nn.Tanh() # testar resultado
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, inputs, debug = False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        batch_size = inputs.shape[0]

        # transformar todos os ids em embeddings
        X = self.embedding_layer(inputs)
        
        # definir entrada residual
        residual = X

        # utilizar somente o último vetor do X como query
        Q = self.W_q(X[:,-1,:].unsqueeze(1)) #.to(device)

        # definir K e V
        K = self.W_k(X) #.to(device)
        V = self.W_v(X) #.to(device)

        # multiplicação matricial entre Q e K.t
        K_transpose = K.transpose(1,2)
        scores = torch.matmul(Q, K_transpose)

        # obter probabilidades 
        probs = self.softmax1(scores)

        # segunda multiplicação matricial
        E = torch.matmul(probs, V)
        E = self.W_0(E)

        # logitos
        logits = self.linear1(E.view(batch_size,-1)) 
        logits = self.relu1(logits)
        logits = self.linear2(logits)
        #logits = self.relu2(logits)

        if debug:
          print(f'X shape: {X.shape}') # [B, L, D]
          print(f'Q shape: {Q.shape}') # [B, 1, D]
          print(f'K shape: {K.shape}') # [B, L, D]
          print(f'K transposed shape: {K_transpose.shape}') # [B, D, L]
          print(f'V shape: {V.shape}') # [B, L, D]
          print(f'scores shape: {scores.shape}') #[B, 1, L]
          print(f'probs shape: {probs.shape}') # [B, 1, L]
          print(f'E shape: {E.shape}') # [B, 1, D]
          print(f'logits shape: {logits.shape}') #[B, V]

        return logits

## Teste o modelo com um exemplo

Não considerei a GPU nesse teste paraeconomizar RAM

In [ ]:
# sem gpu

embedding_dim = 10
context_size = 9
batch_size = 2

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=embedding_dim,
)

sample_train, sample_target = next(iter(DataLoader(training_dataset, batch_size=batch_size)))
sample_train_gpu = sample_train
model_test = model(sample_train_gpu, debug = True)
model_test_shape = model_test.shape

print(f'batch size: {batch_size}')
print(f'sample train: {sample_train}')
print(f'sample target: {sample_target}')
print(f'model test shape: {model_test_shape}')
print(f'model test: {model_test}')

X shape: torch.Size([2, 9, 10])
Q shape: torch.Size([2, 1, 10])
K shape: torch.Size([2, 9, 10])
K transposed shape: torch.Size([2, 10, 9])
V shape: torch.Size([2, 9, 10])
scores shape: torch.Size([2, 1, 9])
probs shape: torch.Size([2, 1, 9])
E shape: torch.Size([2, 1, 10])
logits shape: torch.Size([2, 29794])
batch size: 2
sample train: tensor([[20100,  2308,  3074,  1089,   481,   117,   146,  1189,   125],
        [ 2308,  3074,  1089,   481,   117,   146,  1189,   125, 13254]])
sample target: tensor([13254,   143])
model test shape: torch.Size([2, 29794])
model test: tensor([[0.0876, 0.0000, 0.0462,  ..., 0.1506, 0.1070, 0.0000],
        [0.1067, 0.0000, 0.0557,  ..., 0.1538, 0.1054, 0.0000]],
       grad_fn=<ReluBackward0>)


## Assert da Perplexidade


In [9]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)

Fazer a verificação da PPL sem considerar a GPU também. 

In [ ]:
# sem gpu

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train
target_token_ids = target_token_ids
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30031
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

Considerar a GPU somente nesta parte. Além disso, diminuir o tamanho do batch.

In [15]:
# com gpu
# laço com save 

max_examples = 400_000_000
eval_every_steps = 10000
lr = 3e-4
compare=float('inf')

embedding_dim = 512 # testar com 256
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=embedding_dim,
).to(device)

batch_size = 2048
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=batch_size)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/"+f"model_{max_examples/1000000}ex_{embedding_dim}embdim.pt")
                    with open("/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/"+f"model_{max_examples/1000000}ex_{embedding_dim}embdim.txt", 'w') as f:
                      lines = [f'batch size = {batch_size}', 
                                f'embedding dim = {embedding_dim}', 
                                f'max examples = {max_examples}', 
                                f'learning rate = {lr}', 
                                f'context size = {context_size}', 
                                f'train PPL = {train_ppl}',
                                f'validation PPL = {valid_ppl}',
                                f'best values at {n_examples} examples']
                      f.writelines('\n'.join(lines))
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29778.37, valid ppl: 29698.67
10000 steps; 20480000 examples so far; train ppl: 805.32, valid ppl: 381.43
20000 steps; 40960000 examples so far; train ppl: 326.65, valid ppl: 291.65
30000 steps; 61440000 examples so far; train ppl: 280.85, valid ppl: 270.35
40000 steps; 81920000 examples so far; train ppl: 265.63, valid ppl: 261.06
50000 steps; 102400000 examples so far; train ppl: 255.37, valid ppl: 255.29
60000 steps; 122880000 examples so far; train ppl: 250.31, valid ppl: 251.93
70000 steps; 143360000 examples so far; train ppl: 246.90, valid ppl: 249.77
80000 steps; 163840000 examples so far; train ppl: 243.86, valid ppl: 247.49
90000 steps; 184320000 examples so far; train ppl: 240.42, valid ppl: 246.13
100000 steps; 204800000 examples so far; train ppl: 239.23, valid ppl: 246.39
110000 steps; 225280000 examples so far; train ppl: 238.23, valid ppl: 245.08
120000 steps; 245760000 examples so far; train ppl: 236.51, valid ppl: 243.98
130000 s

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [16]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 230.98233269318882


400_000_000 -> test PPL = 230,9823, val PPL = 241,10, train PPL = 231,33. 

Houve a convergência do modelo, mas não houve a geração de frases interessantes. 

# Continuar o treinamento de um modelo armazenado

In [ ]:
# laço para ocntinuar treinando um modelo salvo

max_examples = 400_000_000
eval_every_steps = 10000
lr = 3e-4
compare=float('inf')

# definir o caminho do modelo a ser considerado
model = torch.load("/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/model_400.0ex_512embdim.pt", map_location=device)
model.to(device)

batch_size = 2048
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=batch_size)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/"+'continuar_treinamento_'+f"model_{max_examples/1000000}Mex_{embedding_dim}embdim.pt")
                    with open("/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/"+'continuar_treinamento_'+f"model_{max_examples/1000000}Mex_{embedding_dim}embdim.txt", 'w') as f:
                      lines = [f'batch size = {batch_size}', 
                                f'embedding dim = {embedding_dim}', 
                                f'max examples = {max_examples}', 
                                f'learning rate = {lr}', 
                                f'context size = {context_size}', 
                                f'train PPL = {train_ppl}',
                                f'validation PPL = {valid_ppl}',
                                f'best values at {n_examples} examples']
                      f.writelines('\n'.join(lines))
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 216.61, valid ppl: 241.17
10000 steps; 20480000 examples so far; train ppl: 233.97, valid ppl: 241.96
20000 steps; 40960000 examples so far; train ppl: 231.80, valid ppl: 242.22
30000 steps; 61440000 examples so far; train ppl: 230.69, valid ppl: 241.76
40000 steps; 81920000 examples so far; train ppl: 230.60, valid ppl: 241.02
50000 steps; 102400000 examples so far; train ppl: 229.33, valid ppl: 240.38
60000 steps; 122880000 examples so far; train ppl: 228.93, valid ppl: 240.24


# Utilizar o melhor modelo encontrado

Como houve problema com o tempo de execução e uso da GPU disponibilizada pelo Colab, irei utilizar o melhor modelo encontrado no último treinamento. 

In [15]:
model = torch.load("/content/drive/MyDrive/Intro ao Aprendizado Profundo/Trabalho08/modelos_salvos/continuar_treinamento_model_400.0Mex_512embdim.pt", map_location=torch.device('cpu'))
model

LanguageModel(
  (embedding_layer): Embedding(29794, 512)
  (W_q): Linear(in_features=512, out_features=512, bias=False)
  (W_k): Linear(in_features=512, out_features=512, bias=False)
  (W_v): Linear(in_features=512, out_features=512, bias=False)
  (W_0): Linear(in_features=512, out_features=512, bias=False)
  (softmax1): Softmax(dim=-1)
  (softmax2): Softmax(dim=-1)
  (linear1): Linear(in_features=512, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=29794, bias=False)
  (tanh1): Tanh()
  (relu1): ReLU()
  (relu2): ReLU()
)

In [17]:
test_loader = DataLoader(test_dataset, batch_size=64)


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input, target)
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 230.76655109614148


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [19]:
prompt = 'Eu gosto de comer pizza pois me faz parte'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte do
Eu gosto de comer pizza pois me faz parte do que
Eu gosto de comer pizza pois me faz parte do que o
Eu gosto de comer pizza pois me faz parte do que o que
Eu gosto de comer pizza pois me faz parte do que o que o
Eu gosto de comer pizza pois me faz parte do que o que o que
Eu gosto de comer pizza pois me faz parte do que o que o que o
Eu gosto de comer pizza pois me faz parte do que o que o que o que
Eu gosto de comer pizza pois me faz parte do que o que o que o que o
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que o
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que o que
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que o que o
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que o que o que
Eu gosto de comer pizza pois me faz parte do que o que o que o que o que o que o que o
Eu 

In [31]:
prompt = 'O tráfico de drogas deve ser combatido pelas forças'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

O tráfico de drogas deve ser combatido pelas forças de
O tráfico de drogas deve ser combatido pelas forças de um
O tráfico de drogas deve ser combatido pelas forças de um dos
O tráfico de drogas deve ser combatido pelas forças de um dos seus
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos,
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que o
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que o que
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que o que o
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que o que o que
O tráfico de drogas deve ser combatido pelas forças de um dos seus filhos, o que o que o que o
O tr

In [33]:
prompt = 'O estudo de redes neurais é importante para produzir'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

O estudo de redes neurais é importante para produzir um
O estudo de redes neurais é importante para produzir um dos
O estudo de redes neurais é importante para produzir um dos seus
O estudo de redes neurais é importante para produzir um dos seus filhos
O estudo de redes neurais é importante para produzir um dos seus filhos,
O estudo de redes neurais é importante para produzir um dos seus filhos, o
O estudo de redes neurais é importante para produzir um dos seus filhos, o que
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o que
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o que o
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o que o que
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o que o que o
O estudo de redes neurais é importante para produzir um dos seus filhos, o que o qu

# Implementação em forma de matrizes sem usar projeções lineares

In [ ]:
# implementação com matrizes

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        self.embedding_layer = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)

        # linear projections
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_0 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        # definir a softmax
        self.softmax1 = nn.Softmax(dim=-1)
        self.softmax2 = nn.Softmax(dim=-1)

        # camada linear - aplicar somente uma camada
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        self.tanh1 = nn.Tanh() # testar resultado
        self.relu1 = nn.ReLU()

    def forward(self, inputs, debug = False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        batch_size = inputs.shape[0]

        # transformar todos os ids em embeddings
        X = self.embedding_layer(inputs)

        # utilizar somente o último vetor do X como query
        Q = X[:,-1]

        # definir K e V
        K = X
        V = X

        # multiplicação matricial entre Q e K.t
        K_transpose = K.transpose(1,2)
        Q_unsqueeze = Q.unsqueeze(1)
        scores = torch.matmul(Q_unsqueeze, K_transpose)

        # obter probabilidades 
        probs = self.softmax1(scores)

        # segunda multiplicação matricial
        E = torch.matmul(probs, V)

        # logitos
        logits = self.linear1(E[:,-1])
        logits = self.relu1(logits)

        if debug:
          print(f'X shape: {X.shape}') # [B, L, D]
          print(f'Q shape: {Q.shape}') # [B, D]
          print(f'Q unsqueeze shape: {Q_unsqueeze.shape}')  #[B, 1, D]
          print(f'K shape: {K.shape}') # [B, L, D]
          print(f'K transposed shape: {K_transpose.shape}') # [B, D, L]
          print(f'scores shape: {scores.shape}') #[D, 1, L]
          print(f'V shape: {V.shape}') # [B, L, D]
          print(f'probs shape: {probs.shape}') # [B, 1, L]
          print(f'E shape: {E.shape}') # [B, 1, D]
          print(f'logits shape: {logits.shape}') #[2, V]

        return logits

# Implementação em forma de loop sem utilizar projeções lineares

In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        self.embedding_layer = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)

        # definir a softmax
        self.softmax1 = nn.Softmax(dim=-1)
        self.softmax2 = nn.Softmax(dim=-1)

        # camada linear
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        self.tanh1 = nn.Tanh()
        self.relu1 = nn.ReLU()

    def forward(self, inputs, debug = False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        logits = []
        for input in inputs: 
          q = input[-1]
          C_q = self.embedding_layer(q)

          scores = []
          for k in input:
            K_emb = self.embedding_layer(k)
            score = torch.matmul(C_q, K_emb.t())
            scores.append(score)

          scores = torch.Tensor(scores)

          probs = self.softmax1(scores)

          E = 0
          for v, p in zip(input, probs):
            E = E + self.embedding_layer(v) * (p)

          logits_input = self.linear1(E)
          logits_input = self.relu1(logits_input)
          logits.append(logits_input)

        batch_size = inputs.shape[0]

        logits = torch.stack(logits, dim=0)
        logits = logits.view(batch_size, self.vocab_size)

        if debug:
          print(f'scores: {scores}')
          print(f'probs: {probs}')
          print(f'E:{E}')
          print(f'logits shape: {logits.shape}')

        return logits

# Rascunho

Algumas realizações foram necessárias para implementar o modelo. Alguns desses casos estão anotados na seguinte célula. 

In [ ]:
#train_loader = DataLoader(training_dataset, batch_size=64, shuffle=False, drop_last=True)

#sample_train, sample_target = next(iter(DataLoader(training_dataset)))

print(f'treino: {sample_train}')
print(f'target: {sample_target}')

vocab_size = tokenizer.vocab_size
embedding_dim = 6
embedding_layer = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)
q = sample_train[:,-1]
C_q = embedding_layer(q)

print(f'w_q: {q}')
print(f'C_q: {C_q}')

input = sample_train
scores = []
for k in input:
  K_emb = embedding_layer(k)
  print(f'k_emb:{K_emb}')
  score = torch.matmul(C_q, K_emb.t())
  scores.append(score)

print(f'scores: {scores}')

scores_out_list = scores[0]

#context_size = 9
#softmax = nn.Softmax(dim=(context_size+1))
probs = nn.functional.softmax(scores_out_list)

print(f'probs: {probs}')

E = 0
i=0
for v, p in torch.stack((sample_train, probs), dim=1):
  for i in range(0,(sample_train.shape[1])):
    #print(f'v:{v[i]}')
    #print(f'p:{p[i]}')
    E = E + embedding_layer(torch.tensor(v[i], dtype=torch.long)) * (p[i])
    i+=1

print(f'E:{E}')

hidden_size = 128
linear1 = nn.Linear(len(E), hidden_size * 2)
logits = linear1(E)
print(f'logits: {logits}')

#softmax = nn.Softmax(dim=0)
probs = nn.functional.softmax(logits)
print(f'probs after logits: {probs}')

token_id_found = probs.argmax()
print(f'token id found: {token_id_found}')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
